importing needed libraries (tensorflow and matplotlib

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

Dataset used in project is download from http://ai.stanford.edu/~btaskar/ocr/ and converted to csv file type. Dataset contains handwritten words, which have been divided into letters, collected by Rob Kassel at MIT Spoken Language Systems Group.

Data contains fields:

1. id: each letter is assigned a unique integer id
2. letter: a-z
3. next_id: id for next letter in the word, -1 if last letter
4. word_id: each word is assigned a unique integer id (not used)
5. position: position of letter in the word (not used)
6. fold: 0-9 -- cross-validation fold
7. p_i_j: 0/1 -- value of pixel in row i, column j